In [132]:
import numpy as np
import pandas as pd

### Load parameter

In [133]:
sheet = "Problem 3 Instance 2"
file = "OR112-2_hw02_data.xlsx"

m = pd.read_excel(file,usecols="B:B",nrows=1,sheet_name=sheet)
m = int(m.columns.item())
n = pd.read_excel(file,usecols="B:B",skiprows=range(1),nrows=1,sheet_name=sheet)
n = int(n.columns.item())
p = pd.read_excel(file,usecols="B:B",skiprows=range(2),nrows=1,sheet_name=sheet)
p = int(p.columns.item())

d = pd.read_excel(file,usecols="E:"+chr(ord('E')+n-1),skiprows=range(4),nrows=m,sheet_name=sheet)
d = d.to_numpy()

h = pd.read_excel(file,usecols="B:B",skiprows=range(4),nrows=m,sheet_name=sheet)
h = h.to_numpy()

## Algorithm
### Greedy

In [134]:
def objfunc(h,d,x):
    total = 0
    for i in range(h.shape[0]):
        total += h[i] * min(d[i,x.astype(bool)])
    return total.item()



In [135]:
x = np.ones(n)
for i in range(n-p):
    total = np.zeros(n)
    for j in range(n-i):
        tmp_x = x.copy()
        if tmp_x[j] == 1:
            tmp_x[j] = 0
            total[j] = objfunc(h,d,tmp_x)
    x[np.argmax(total)] = 0
print(x)
print(objfunc(h,d,x))


[0. 1. 0. 1. 0. 1. 0. 0. 1. 1.]
65364


In [136]:
for i in range(n):
    print(f"x_{i+1} = {int(x[i])}", end=', ')
print()
for j in range(m):
    print(f"w_{j+1} = {min(d[j,x.astype(bool)])}", end=', ')

x_1 = 0, x_2 = 1, x_3 = 0, x_4 = 1, x_5 = 0, x_6 = 1, x_7 = 0, x_8 = 0, x_9 = 1, x_10 = 1, 
w_1 = 48, w_2 = 37, w_3 = 67, w_4 = 45, w_5 = 87, w_6 = 55, w_7 = 76, w_8 = 108, w_9 = 26, w_10 = 107, w_11 = 37, w_12 = 121, w_13 = 29, w_14 = 88, w_15 = 56, w_16 = 85, w_17 = 100, w_18 = 71, w_19 = 7, w_20 = 23, 

### LR

In [137]:
from gurobipy import *
import numpy as np
import pandas as pd

sheet = "Problem 3 Instance 1"
file = "OR112-2_hw02_data.xlsx"
model = Model(sheet)

m = pd.read_excel(file,usecols="B:B",nrows=1,sheet_name=sheet)
m = int(m.columns.item())
n = pd.read_excel(file,usecols="B:B",skiprows=range(1),nrows=1,sheet_name=sheet)
n = int(n.columns.item())
p = pd.read_excel(file,usecols="B:B",skiprows=range(2),nrows=1,sheet_name=sheet)
p = int(p.columns.item())

d = pd.read_excel(file,usecols="E:"+chr(ord('E')+n-1),skiprows=range(4),nrows=m,sheet_name=sheet)
d = d.to_numpy()

h = pd.read_excel(file,usecols="B:B",skiprows=range(4),nrows=m,sheet_name=sheet)
h = h.to_numpy()

x = model.addVars(n,lb=0.0,ub=1.0,vtype=GRB.CONTINUOUS, name="x")
w = model.addVars(m,vtype=GRB.CONTINUOUS, name="w")

model.setObjective(quicksum(h[i][0]*w[i] for i in range(m)), GRB.MAXIMIZE)

model.addConstr(quicksum(x[i] for i in range(n)) >= p)
for i in range(m):
    for j in range(n):
        model.addConstr(w[i] <= d[i][j]*x[j]+np.max(d[i])*(1-x[j]))

model.optimize()
x = []
# for v in model.getVars():
#     print('%s %g' % (v.varName, v.x))
#     x.append(v.x)

for v in model.getVars():
    print(f"&{v.varName.replace('[','_{').replace(']','}')} = {v.x}", end=',\\\\ ')
    x.append(v.x)

x = x[:n]

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 22.6.0 22G91)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 51 rows, 15 columns and 95 nonzeros
Model fingerprint: 0xf307f753
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+01, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 2e+02]
Presolve removed 10 rows and 0 columns
Presolve time: 0.01s
Presolved: 41 rows, 15 columns, 85 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.2847000e+04   4.800000e+01   0.000000e+00      0s
      21    5.3434867e+04   0.000000e+00   0.000000e+00      0s

Solved in 21 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.343486724e+04
&x_{0} = 0.5949548088466156,\\ &x_{1} = 1.0,\\ &x_{2} = 0.7487071751777633,\\ &x_{3} = 0.38062725090036004,\\ &x_{4} = 0.2757107650752609,\\ &w_{0} = 100.89225690276112,\\ &

In [138]:
tmp_x = np.zeros(n)
print(x)
for i in range(p):
    tmp_x[np.argmax(x)] = 1
    x[np.argmax(x)] = 0

for i in range(n):
    print(f"&x_{i+1} = {int(tmp_x[i])}", end=',\\\\\n ')
for i in range(m):
    print(f"w_{{{i+1}}} = {min(d[i,tmp_x.astype(bool)])}", end=', ')
print()
print(objfunc(h,d,tmp_x))

[0.5949548088466156, 1.0, 0.7487071751777633, 0.38062725090036004, 0.2757107650752609]
&x_1 = 1,\\
 &x_2 = 1,\\
 &x_3 = 1,\\
 &x_4 = 0,\\
 &x_5 = 0,\\
 w_{1} = 76, w_{2} = 108, w_{3} = 30, w_{4} = 107, w_{5} = 66, w_{6} = 121, w_{7} = 29, w_{8} = 95, w_{9} = 56, w_{10} = 30, 
39027


In [148]:
a = 117810.4228
b = 65364

print((a-b)/a)
print(pow(500,3))
print(pow(100,4)*500)

0.4451764245769263
125000000
50000000000
